## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-01-18-57-19 +0000,nypost,Stunning number of California voters back weal...,https://nypost.com/2026/02/01/us-news/stunning...
1,2026-02-01-18-54-51 +0000,wapo,ESPN closes deal for NFL Network following reg...,https://www.washingtonpost.com/business/2026/0...
2,2026-02-01-18-45-43 +0000,nypost,Boozed-up whacko strips down on Universal Stud...,https://nypost.com/2026/02/01/us-news/man-at-u...
3,2026-02-01-18-28-58 +0000,nyt,What to Know About the Rafah Border Crossing i...,https://www.nytimes.com/2026/02/01/world/middl...
4,2026-02-01-18-28-27 +0000,nyt,Judge Who Ruled Against Trump Administration C...,https://www.nytimes.com/2026/02/01/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2391/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,24
84,ice,15
46,epstein,13
86,minnesota,10
47,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
121,2026-02-01-04-26-00 +0000,wsj,President Trump joked during a speech that he ...,https://www.wsj.com/politics/policy/trump-joke...,67
46,2026-02-01-15-09-48 +0000,nypost,Trump jokes about suing new Fed chair nominee ...,https://nypost.com/2026/02/01/us-news/trump-jo...,61
166,2026-01-31-20-52-55 +0000,cbc,"Judge orders U.S. to free 5-year-old boy, fath...",https://www.cbc.ca/news/world/father-son-relea...,60
67,2026-02-01-13-05-34 +0000,nypost,"Trump threatens to sue Michael Wolff, Epstein ...",https://nypost.com/2026/02/01/us-news/trump-th...,54
132,2026-02-01-02-00-00 +0000,wsj,An Abu Dhabi royal backed a secret $500 millio...,https://www.wsj.com/politics/policy/spy-sheikh...,53


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
121,67,2026-02-01-04-26-00 +0000,wsj,President Trump joked during a speech that he ...,https://www.wsj.com/politics/policy/trump-joke...
166,60,2026-01-31-20-52-55 +0000,cbc,"Judge orders U.S. to free 5-year-old boy, fath...",https://www.cbc.ca/news/world/father-son-relea...
63,45,2026-02-01-13-52-32 +0000,nypost,Ex-Prince Andrew’s creepy reply to Jeffrey Eps...,https://nypost.com/2026/02/01/world-news/ex-pr...
132,29,2026-02-01-02-00-00 +0000,wsj,An Abu Dhabi royal backed a secret $500 millio...,https://www.wsj.com/politics/policy/spy-sheikh...
57,25,2026-02-01-14-18-00 +0000,wsj,Gaza Crossing Into Egypt Will Open to Palestin...,https://www.wsj.com/world/middle-east/gaza-cro...
89,24,2026-02-01-10-54-14 +0000,nypost,Iran’s supreme leader warns any US attack woul...,https://nypost.com/2026/02/01/world-news/iran-...
149,23,2026-01-31-23-08-20 +0000,bbc,"Andrew should testify to US Congress, Starmer ...",https://www.bbc.com/news/articles/cgrd7kpg9pwo...
29,22,2026-02-01-17-10-59 +0000,nypost,First lady’s film ‘Melania’ racks up biggest o...,https://nypost.com/2026/02/01/us-news/first-la...
85,21,2026-02-01-11-00-00 +0000,latimes,Chabria: A California lawyer takes the civil r...,https://www.latimes.com/california/story/2026-...
173,21,2026-01-31-20-17-52 +0000,latimes,Man sentenced for throwing Molotov cocktail at...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
